In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import random as random
import tensorflow.compat.v1 as tf
import time


def generate_user_location(number_samples, num_users,location_user):
    x_wall = [24, 0]  # X=10 X=12
    y_wall = [12, -12]  # X=10 X=12
    RIS_wall = 0  # ris in x

    x=location_user[:, :, :, :, 0:1]
    y=location_user[:, :, :, :, 1:2]
    z=location_user[:, :, :, :, 2:3]
    Refl1_user = np.zeros_like(location_user)
    Refl1_ris_user = np.zeros_like(location_user)
    Refl2_user = np.zeros_like(location_user)
    Refl2_ris_user = np.zeros_like(location_user)
    #########reflect once########################
    for refwall in [i * 2 for i in y_wall]:
        temp = np.concatenate((x, refwall - y, z), axis=4)
        Refl1_user = np.concatenate((Refl1_user, temp), axis=1)
        Refl1_ris_user = np.concatenate((Refl1_ris_user, temp), axis=1)
    for refwall in [i * 2 for i in x_wall]:
        temp = np.concatenate((refwall - x, y, z), axis=4)
        Refl1_user = np.concatenate((Refl1_user, temp), axis=1)
        if refwall == RIS_wall * 2:
            continue
        else:
            Refl1_ris_user = np.concatenate((Refl1_ris_user, temp), axis=1)

    Refl1_user = Refl1_user[:, 1:, :, :, :]  # remove the all zeros line
    Refl1_ris_user = Refl1_ris_user[:, 1:, :, :, :]
    #########reflect twice########################
    for ref1 in [i * 2 for i in y_wall]:
        ref2 = sum(y_wall) * 2 - ref1
        temp = np.concatenate((x, ref2 - (ref1 - y), z), axis=4)
        Refl2_user = np.concatenate((Refl2_user, temp), axis=1)
        Refl2_ris_user = np.concatenate((Refl2_ris_user, temp), axis=1)
    for ref1 in [i * 2 for i in x_wall]:
        ref2 = sum(x_wall) * 2 - ref1
        temp = np.concatenate((ref2 - (ref1 - x), y, z), axis=4)
        Refl2_user = np.concatenate((Refl2_user, temp), axis=1)
        if ref2 == RIS_wall * 2:
            continue
        else:
            Refl2_ris_user = np.concatenate((Refl2_ris_user, temp), axis=1)
    for ref1 in [i * 2 for i in x_wall]:
        for ref2 in [i * 2 for i in y_wall]:
            # print(ref1,ref2)
            temp = np.concatenate((ref1 - x, ref2 - y, z), axis=4)
            Refl2_user = np.concatenate((Refl2_user, temp), axis=1)
            if ref1 == RIS_wall * 2:
                # print('12334',ref1,ref2)
                continue
            else:
                # print('2 \n',np.shape(Refl2_ris_user))
                Refl2_ris_user = np.concatenate((Refl2_ris_user, temp), axis=1)
    Refl2_user = Refl2_user[:, 1:, :, :, :]  # remove the all zeros line
    Refl2_ris_user = Refl2_ris_user[:, 1:, :, :, :]  # remove the all zeros line
    # print(np.shape(location_user),np.shape(Refl1_user),np.shape(Refl2_user),np.shape(Refl1_ris_user),np.shape(Refl2_ris_user))
    #print(Refl1_user)
    return location_user, Refl1_user, Refl2_user, Refl1_ris_user, Refl2_ris_user


def generate_irs_location(number_samples, num_irs=2):
    x = np.reshape(np.array([0.0]), [1, 1, -1, 1, 1])[:, :, :num_irs, :, :]
    y = np.reshape(np.array([0.0]), [1, 1, -1, 1, 1])[:, :, :num_irs, :, :]
    z = 0.0 * np.ones_like(x)
    location_bs = np.concatenate((x, y, z), axis=4) * np.ones([number_samples, 1, num_irs, 1, 3])  # (num_bs,3)
    return location_bs




def generate_bs_location(number_samples, num_BS):
    x = np.reshape(np.array([12]), [1, 1, 1, 1, 1])
    y = np.reshape(np.array([-11.5]), [1, 1, 1, 1, 1])
    z = 0.0 * np.ones_like(x)
    location_bs = np.concatenate((x, y, z), axis=4) * np.ones([number_samples, 1, 1, num_BS, 3])  # (num_bs,3)
    return location_bs




def cal_angle_distance2(location1, location2):
    distance = np.linalg.norm(location1 - location2, axis=4, keepdims=True)
    aoa_x = (location1[:, :, :, :, 0:1] - location2[:, :, :, :, 0:1]) / distance
    aoa_y = (location1[:, :, :, :, 1:2] - location2[:, :, :, :, 1:2]) / distance
    aoa_z = (location1[:, :, :, :, 2:3] - location2[:, :, :, :, 2:3]) / distance
    return aoa_x, aoa_y, aoa_z

    #
def generate_channel_fullRician(params_system,  num_samples, pathloss_irs_bs,channel_irs_user, testing=0,irs_Nh=16):
    (num_bs, number_irs, num_elements_irs, num_user) = params_system
    location_irs = generate_irs_location(1, number_irs)
    #print(np.shape(location_user),'1111111111111111111111111111111111111')
    location_bs = generate_bs_location(1, num_bs)
    aod_irs_x,aod_irs_y,aod_irs_z=cal_angle_distance2(location_bs,location_irs)
    #################   channel between IRS and BS  ########################

    i1 = np.mod(np.arange(num_elements_irs), irs_Nh).reshape([1, 1, 1, num_elements_irs, 1])
    i2 = np.floor(np.arange(num_elements_irs) / irs_Nh).reshape([1, 1, 1, num_elements_irs, 1])

    pathloss_irs_bs = pathloss_irs_bs*np.ones([num_samples, 1, number_irs, 1, num_bs])
    a_irs_bs = np.exp(1j * np.pi * (i1 * aod_irs_y + i2 * aod_irs_z)).reshape(1, 1, number_irs,
                                                                              num_elements_irs, num_bs)
    channel_bs_irs = (a_irs_bs.conjugate() * pathloss_irs_bs)

    ###############################################################################################
    channel_bs_irs_user = channel_bs_irs * channel_irs_user

    #channel_bs_user= np.sum(channel_bs_user, 1, keepdims=True)

    channels =channel_bs_irs_user

    aoa_aod = (aod_irs_x,aod_irs_y, aod_irs_z )
    return channels, aoa_aod





import tensorflow.compat.v1 as tf
import scipy.io
tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense
# from wsr.bcd.generate_received_pilots import generate_received_pilots_batch


# these three lines are needed if using google colab, otherwise can delete



'System Information'
N = 1  # Number of BS's antennas
delta_inv = 128  # Number of posterior intervals inputed to DNN


tau = 2  # Pilot length
scale_factor=100
snr_const1 = 25  # The SNR
snr_const = np.array([snr_const1])
Pvec = 10 ** (snr_const / 10)  # Set of considered TX powers

mean_true_alpha = 0.0 + 0.0j  # Mean of the fading coefficient
std_per_dim_alpha = np.sqrt(0.5)  # STD of the Gaussian fading coefficient per real dim.
noiseSTD_per_dim = np.sqrt(0.5)  # STD of the Gaussian noise per real dim.
#####################################################
'RIS'
num_ris = 1
N_ris = 64
num_users = 1

location_user = None
N_base = 1
params_system = (N_base, num_ris, N_ris, num_users)  # (num_bs, num_elements_irs, num_user)
input_mode = 1
#####################################################
'Learning Parameters'
batch_size_order = 8  # Mini_batch_size = batch_size_order*delta_inv

train_num=batch_size_order * delta_inv
initial_run = 1  # 0: Continue training; 1: Starts from the scratch
if initial_run == 1:
    print('!!!!! training from scratch !!!!!')
n_epochs = 1000  # Num of epochs
learning_rate = 0.00005  # Learning rate
batch_per_epoch = 100  # Number of mini batches per epoch

val_size_order = 8  # Validation_set_size = val_size_order*delta_inv
test_size_order = 782  # Test_set_size = test_size_order*delta_inv*scale_factor
######################################################
tf.reset_default_graph()  # Reseting the graph
he_init = tf.variance_scaling_initializer()  # Define initialization method
######################################## Place Holders
# alpha_input = tf.placeholder(tf.float32, shape=(None,1), name="alpha_input")

# alpha_input = tf.placeholder(tf.float32, shape=(None,1), name="alpha_input")
loc_input = tf.placeholder(tf.float32, shape=(None, 3), name="loc_input")

channel_bs_irs_user = tf.placeholder(tf.complex64, shape=(None, num_users, num_ris, N_ris, 1), name="channel_irs_user")

channel_bs_irs_user2=tf.reshape(channel_bs_irs_user,(-1,1, num_users, num_ris, N_ris, 1))

channel_bs_user = tf.placeholder(tf.complex64, shape=(None,   N_base), name="channel_bs_user")

beamform_time= int(18/tau)
#########################################################################################################################
# Constructing the array responses for AoA samples
##################### NETWORK############################################################################################
with tf.name_scope("array_response_construction"):
    lay = {}

    lay['P'] = tf.constant(1.0)*Pvec
    ###############
    from0toN = tf.cast(tf.range(0, N, 1), tf.float32)

with tf.name_scope("channel_sensing"):
    hidden_size = 431
    A1 = tf.get_variable("A1", shape=[hidden_size, hidden_size*2], dtype=tf.float32, initializer=he_init)
    A2 = tf.get_variable("A2", shape=[hidden_size*2, hidden_size*2], dtype=tf.float32, initializer=he_init)
    A3 = tf.get_variable("A3", shape=[hidden_size*2, hidden_size*2], dtype=tf.float32, initializer=he_init)
    A4 = tf.get_variable("A4", shape=[hidden_size*2, 2 * N_ris * num_ris*beamform_time], dtype=tf.float32, initializer=he_init)

    b1 = tf.get_variable("b1", shape=[hidden_size*2], dtype=tf.float32, initializer=he_init)
    b2 = tf.get_variable("b2", shape=[hidden_size*2], dtype=tf.float32, initializer=he_init)
    b3 = tf.get_variable("b3", shape=[hidden_size*2], dtype=tf.float32, initializer=he_init)
    b4 = tf.get_variable("b4", shape=[2 * N_ris * num_ris*beamform_time], dtype=tf.float32, initializer=he_init)

    w_dict = []
    posterior_dict = []
    idx_est_dict = []
    layer_Ui = Dense(units=hidden_size, activation='linear')
    layer_Wi = Dense(units=hidden_size, activation='linear')
    layer_Uf = Dense(units=hidden_size, activation='linear')
    layer_Wf = Dense(units=hidden_size, activation='linear')
    layer_Uo = Dense(units=hidden_size, activation='linear')
    layer_Wo = Dense(units=hidden_size, activation='linear')
    layer_Uc = Dense(units=hidden_size, activation='linear')
    layer_Wc = Dense(units=hidden_size, activation='linear')


    def RNN(input_x, h_old, c_old):
        i_t = tf.sigmoid(layer_Ui(input_x) + layer_Wi(h_old))
        f_t = tf.sigmoid(layer_Uf(input_x) + layer_Wf(h_old))
        o_t = tf.sigmoid(layer_Uo(input_x) + layer_Wo(h_old))
        c_t = tf.tanh(layer_Uc(input_x) + layer_Wc(h_old))
        c = i_t * c_t + f_t * c_old  # cell state
        h_new = o_t * tf.tanh(c)  # hidden state
        return h_new, c


    snr = lay['P'] * tf.ones(shape=[tf.shape(loc_input)[0], 1], dtype=tf.float32)

    all_output = tf.zeros(shape=[tf.shape(loc_input)[0], 1], dtype=tf.float32)
    all_output2 = tf.zeros(shape=[tf.shape(loc_input)[0], 1], dtype=tf.float32)
    y_real_all=tf.zeros(shape=[tf.shape(loc_input)[0], beamform_time], dtype=tf.float32)
    theta_final=tf.zeros(shape=[tf.shape(loc_input)[0],beamform_time, N_ris * num_ris], dtype=tf.complex64)
    for t in range(tau):
        'DNN designs the next sensing direction'
        if t == 0:
            RSS_val = tf.ones([tf.shape(loc_input)[0], beamform_time*2])
            Dif_y = tf.zeros([tf.shape(loc_input)[0], beamform_time*2])
            h_old = tf.zeros([tf.shape(loc_input)[0], hidden_size])
            c_old = tf.zeros([tf.shape(loc_input)[0], hidden_size])
        # print(tf.shape(RSS_val))


        if input_mode==2:
          h_old, c_old = RNN(tf.concat([RSS_val,Dif_y],axis=1), h_old, c_old)
        else:
          h_old, c_old = RNN(tf.concat([RSS_val],axis=1), h_old, c_old)
        x1 = tf.nn.relu(h_old @ A1 + b1)
        x1 = BatchNormalization()(x1)
        x2 = tf.nn.relu(x1 @ A2 + b2)
        x2 = BatchNormalization()(x2)
        x3 = tf.nn.relu(x2 @ A3 + b3)
        x3 = BatchNormalization()(x3)
        '''
            RIS implementation
        '''
        ris_her_unnorm = x3 @ A4 + b4
        ris_her_unnorm = tf.reshape(ris_her_unnorm,[-1,beamform_time,2 * N_ris * num_ris])
        ris_her_r = ris_her_unnorm[:,:, 0:N_ris * num_ris]
        ris_her_i = ris_her_unnorm[:,:, N_ris * num_ris:2 * N_ris * num_ris]  # (? , N_ris)
        theta_tmp = tf.sqrt(tf.square(ris_her_r) + tf.square(ris_her_i))  # (? , N_ris)
        theta_real = ris_her_r / theta_tmp  # (? , N_ris)
        theta_imag = ris_her_i / theta_tmp  # (? , N_ris)
        #theta_temp=tf.concat([ theta_real[:,:] , theta_imag[:,:]], axis=1)
        theta_temp=tf.complex(theta_real, theta_imag)
        theta_final=  tf.concat([theta_final ,theta_temp], axis=0)
        #print(theta)                     #  (?, 1, 128)
        theta_real = tf.reshape(theta_real, [-1,beamform_time, 1, num_ris, N_ris, 1])
        theta_imag = tf.reshape(theta_imag, [-1,beamform_time, 1, num_ris, N_ris, 1])
        'BS observes the next measurement'
        theta=tf.complex(theta_real,theta_imag)
        h_d_plus_h_cas = tf.reduce_sum(channel_bs_irs_user2 * theta,
                                   (2, 3,4))  # since
        h_d_plus_h_cas=tf.reshape(h_d_plus_h_cas,[tf.shape(loc_input)[0],beamform_time*1])+channel_bs_user
        #import tensorflow as tf2
        #tf2.random.set_seed(0)
        noise = tf.complex(tf.random_normal(tf.shape(h_d_plus_h_cas), mean=0.0, stddev=noiseSTD_per_dim), \
                           tf.random_normal(tf.shape(h_d_plus_h_cas), mean=0.0, stddev=noiseSTD_per_dim))
        #y_complex = tf.complex(tf.sqrt(lay['P']), 0.0) * h_d_plus_h_cas + noise
        noise1=noise/ tf.complex(tf.sqrt(lay['P']), 0.0)
        #noise1=noise1/1000
        #if t == 0:
         # noise_init = tf.complex(tf.random_normal(tf.shape(h_d_plus_h_cas), mean=0.0, stddev=noiseSTD_per_dim), \
         #                  tf.random_normal(tf.shape(h_d_plus_h_cas), mean=0.0, stddev=noiseSTD_per_dim))
         # y_init=channel_bs_user+noise_init
        y_complex = h_d_plus_h_cas + noise1
        y_real_all= tf.concat([y_real_all,tf.abs(y_complex)],0)
        #Dif_y= tf.concat( [  tf.real(y_complex-y_init ), tf.imag(y_complex-y_init )],axis=1)
        #RSS_val = tf.abs(y_complex*tf.complex(tf.sqrt(lay['P']), 0.0))
        RSS_val =tf.concat( [tf.real(y_complex ), tf.imag(y_complex )],axis=1)
    # h_old, c_old = RNN(tf.concat([RSS_val,Dif_y],axis=1), h_old, c_old)



    h_old, c_old = RNN(tf.concat([RSS_val],axis=1), h_old, c_old)
    loc_hat = Dense(units=3, activation='linear')(c_old)
    y_real_all=y_real_all[1:,:]
    theta_final=theta_final[1:,:,:]

################################### Loss Function##################################################################
a = tf.math.reduce_euclidean_norm(loc_input[:, :] - loc_input[:, :], 1)
b = tf.math.reduce_euclidean_norm(loc_hat - loc_input[:, :], 1)
loss = tf.keras.losses.mean_squared_error(a, b)

####### Optimizer

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss, name="training_op")
init = tf.global_variables_initializer()
saver = tf.train.Saver()
 #########################################################################
# print(wavelength)

power_bs_irs=-72.129+scale_factor
phase_bs_irs=159.36
power_bs_irs=10**(power_bs_irs/10);
pathloss_bs_irs=np.sqrt(power_bs_irs)*np.exp(1j*(phase_bs_irs/180*np.pi))

training=1
from google.colab import drive
drive.mount('/content/drive')

drive_save_path ='/content/drive/MyDrive/data1/'

# Load the .mat file
if training==1:
    Train_location_user = sio.loadmat(drive_save_path+'location_user.mat')['location_user']
    Val_location_user=sio.loadmat(drive_save_path+'location_user_val.mat')['location_user']
    Train_channel_irs_user =(sio.loadmat(drive_save_path+'channel_irs_user_romdp_ref5.mat')['channel_irs_user']).reshape(-1,1,1,64,1)
    Val_channel_irs_user=(sio.loadmat(drive_save_path+'channel_irs_user_val_romdp_ref5.mat')['channel_irs_user']).reshape(-1,1,1,64,1)
    Train_channel_bs_user=sio.loadmat(drive_save_path+'bs_user_variables.mat')['channel_bs_user']
    Val_channel_bs_user=sio.loadmat(drive_save_path+'bs_user_variables_val.mat')['channel_bs_user']
     #(params_system,  num_samples, pathloss_irs_bs,channel_irs_user, testing=0,irs_Nh=16):
    #1, 1, 1, num_elements_irs, 1
    num_val=1000


else:
    Val_location_user=sio.loadmat(drive_save_path+'location_user_test.mat')['location_user']
    Val_channel_irs_user=(sio.loadmat(drive_save_path+'channel_irs_user_test_romdp_ref5.mat')['channel_irs_user']).reshape(-1,1,1,64,1)
    Val_channel_bs_user=sio.loadmat(drive_save_path+'bs_user_variables_test.mat')['channel_bs_user']
    num_val=10000
    #print(Val_location_user)
###########  Validation Setgenerate_channel_fullRician(params_system,multipath,num_samples,wavelength,location_irs=np.array([0, 0, 0]), scale_factor=100,irs_Nh = 16):
channels_val,aoa_aod= generate_channel_fullRician(params_system,  num_val, pathloss_bs_irs,Val_channel_irs_user)
channel_bs_irs_user_val=channels_val
feed_dict_val = {loc_input: np.array(Val_location_user),
                 channel_bs_irs_user: channel_bs_irs_user_val,
                 channel_bs_user:Val_channel_bs_user}
#########################################   Training ##############################################################
Ris = '/RIS' + str(num_ris) + '_mode' + str(input_mode)
import sys
import os

#drive_save_path = './My_code_new/'+'mode'+str(input_mode)

file_name = drive_save_path + "/snr"+str(snr_const1)+"_sf"+str(scale_factor)+"_beam"+str(beamform_time)+"_tau"+str(tau)
if training==1:
    def mkdir(path):
        folder = os.path.exists(path)

        if not folder:  #
            os.makedirs(path)  # makedirs
            print("create new folder")
        else:
            print('exist folder')
    with tf.Session() as sess:
        # if initial_run == 1:
        init.run()
        #saver.restore(sess, file_name + Ris + '/Tau_' + str(tau) + '_snr_' + str(snr_const[0]))
        # else:
        best_loss = 1000000

        print(tf.test.is_gpu_available())  # Prints whether or not GPU is on

        mkdir(file_name)

        for epoch in range(10000):
            batch_iter = 0
            for rnd_indices in range(batch_per_epoch):
                num_samples=batch_size_order * delta_inv
                choice=np.random.randint(10**6, size=[num_samples])
                Train_channel_irs_user_epoch=Train_channel_irs_user[choice,:,:].reshape(num_samples,1,1,64,1)
                channels_train,_= generate_channel_fullRician(params_system,num_samples,pathloss_bs_irs,Train_channel_irs_user_epoch)
                channel_bs_irs_user_train=channels_train

                feed_dict_batch = {loc_input: Train_location_user[choice,:],
                                   channel_bs_irs_user: channel_bs_irs_user_train,
                                   channel_bs_user:Train_channel_bs_user[choice,:]}
                _,train_loss=sess.run([training_op,loss], feed_dict=feed_dict_batch)
                batch_iter += 1

            power_val,loss_val  = sess.run([lay['P'],loss ], feed_dict=feed_dict_val)
            if epoch % 20 == 0:  # Every 10 iterations it checks if the validation performxZxxZZace is improved, then saves parameters
                print('epoch', epoch, '  loss_test:%2.5f' % loss_val, '  best_test:%2.5f' % best_loss,'  sqrt_loss_test:%2.5f' % np.sqrt(loss_val), '  sqrt_best_test:%2.5f' %  np.sqrt(best_loss))
                print('train_loss',train_loss)
                if loss_val < best_loss:
                  save_path = saver.save(sess, file_name + Ris + '/Tau2_' + str(tau) + '_snr_' + str(snr_const[0]))
                  best_loss = loss_val
                if epoch%800==0:
                  save_path = saver.save(sess, file_name + Ris + '/Tau2_' + str(tau) + '_snr_' + str(snr_const[0])+str(epoch))
                with open( file_name + Ris +'/data.txt', 'a') as f:
                  print('epoch', epoch, '  loss_test:%2.5f' % loss_val, '  best_test:%2.5f' % best_loss, '  sqrt_loss_test:%2.5f' % np.sqrt(loss_val), '  sqrt_best_test:%2.5f' %  np.sqrt(best_loss),file = f)
#################################   TEST ######################


import os
import scipy.io as sio



Ris = '/RIS' + str(num_ris) + '_mode' + str(input_mode)

drive_save_path = './My_code/'
with tf.Session() as sess:

    #init.run()
    val_number=10000
    init_num=0
    saver.restore(sess, file_name + Ris + '/Tau2_' + str(tau) + '_snr_' + str(snr_const[0]))


    '''
        RIS implementation
    '''

      # (aoa_bs, aoa_bs_y, aoa_bs_z, aod_irs_y, aod_irs_z, aoa_irs_y, aoa_irs_z)
    #channel_true_test,aoa_aod = generate_channel_fullRician(params_system,val_number-init_num,pathloss_bs_irs,Val_channel_irs_user)
    (aod_irs_x,aod_irs_y, aod_irs_z)= aoa_aod
    #( channel_bs_user_test,channel_irs_user_test, channel_bs_user_test) = channel_true_test
    #(channel_bs_user,channel_irs_user, channel_bs_irs)
    #print('locaion',Val_location_user)

    #(location_user, location_virtual_user, lvu, ris_user1, ris_user2)=location
    #print("aod_irs_y:",aod_irs_y,'\n',"aod_irs_z:",aod_irs_z,'\n',"aoa_irs_y:",aoa_irs_y,'\n',"aoa_irs_z:",aoa_irs_z)
    print("aod_irs_y:",aod_irs_y,'\n',"aod_irs_z:",aod_irs_z,'\n')
    feed_dict_batch = {loc_input: np.array(Val_location_user[init_num:val_number,:]),
                       channel_bs_irs_user: channel_bs_irs_user_val[init_num:val_number,:,:,:,:],
                       channel_bs_user:Val_channel_bs_user[init_num:val_number,:]}
    mse_loss,theta_test,y_real_value,value,noise_value= sess.run([loss,theta_final,y_real_all, h_d_plus_h_cas , noise1 ],feed_dict=feed_dict_batch)
    #print(np.abs(value))
    #print('noise:',np.abs(noise_value))
    #print(y_real_value,"y_real_value")
    location=Val_location_user[init_num:val_number,:]
    location_user, location_virtual_user, lvu, ris_user1, ris_user2=generate_user_location(val_number, val_number-init_num,location.reshape(val_number-init_num, 1, 1, 1, 3))
    data = {'theta': theta_test,'location':location,'location_virtual_user':location_virtual_user,'lvu':lvu,'ris_user1':ris_user1,'ris_user2':ris_user2}
    sio.savemat(file_name + Ris + '/file2_50_nocint.mat', data)
   # print(y_real_value)
   # print(np.mean(theta_test,0))io.s
    print('loss',np.sqrt(mse_loss))
    with open( file_name + Ris +'/data.txt', 'a') as f:
      print('loss',np.sqrt(mse_loss))

Instructions for updating:
non-resource variables are not supported in the long term


!!!!! training from scratch !!!!!


Instructions for updating:
Colocations handled automatically by placer.


Mounted at /content/drive


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True
exist folder
epoch 0   loss_test:157.32361   best_test:1000000.00000   sqrt_loss_test:12.54287   sqrt_best_test:1000.00000
train_loss 159.80054
epoch 20   loss_test:6.90114   best_test:157.32361   sqrt_loss_test:2.62700   sqrt_best_test:12.54287
train_loss 6.7104554
epoch 40   loss_test:3.74470   best_test:6.90114   sqrt_loss_test:1.93512   sqrt_best_test:2.62700
train_loss 4.3305335
epoch 60   loss_test:3.01084   best_test:3.74470   sqrt_loss_test:1.73518   sqrt_best_test:1.93512
train_loss 3.286548
epoch 80   loss_test:2.60674   best_test:3.01084   sqrt_loss_test:1.61454   sqrt_best_test:1.73518
train_loss 2.424601
epoch 100   loss_test:2.47117   best_test:2.60674   sqrt_loss_test:1.57200   sqrt_best_test:1.61454
train_loss 2.3059812
epoch 120   loss_test:2.25527   best_test:2.47117   sqrt_loss_test:1.50176   sqrt_best_test:1.57200
train_loss 1.9938258
epoch 140   loss_test:2.02110   best_test:2.25527   sqrt_loss_test:1.42165   sqrt_best_test:1.50176
train_loss 2.1329215
epoch 1

Instructions for updating:
Use standard file APIs to delete files with this prefix.


epoch 3220   loss_test:0.27972   best_test:0.27008   sqrt_loss_test:0.52889   sqrt_best_test:0.51969
train_loss 0.28259918
epoch 3240   loss_test:0.28382   best_test:0.27008   sqrt_loss_test:0.53274   sqrt_best_test:0.51969
train_loss 0.26855817
epoch 3260   loss_test:0.28273   best_test:0.27008   sqrt_loss_test:0.53173   sqrt_best_test:0.51969
train_loss 0.28028867
epoch 3280   loss_test:0.26150   best_test:0.27008   sqrt_loss_test:0.51137   sqrt_best_test:0.51969
train_loss 0.2684714
epoch 3300   loss_test:0.27119   best_test:0.26150   sqrt_loss_test:0.52076   sqrt_best_test:0.51137
train_loss 0.26454452
epoch 3320   loss_test:0.30079   best_test:0.26150   sqrt_loss_test:0.54844   sqrt_best_test:0.51137
train_loss 0.24834387
epoch 3340   loss_test:0.29658   best_test:0.26150   sqrt_loss_test:0.54459   sqrt_best_test:0.51137
train_loss 0.26530913
epoch 3360   loss_test:0.27867   best_test:0.26150   sqrt_loss_test:0.52789   sqrt_best_test:0.51137
train_loss 0.2671954
epoch 3380   loss_

KeyboardInterrupt: ignored